This notebook demonstrates a test of autocnet on the New Horizons LORRI data

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

import unittest

from autocnet.examples import get_path
from autocnet.fileio.io_controlnetwork import to_isis
from autocnet.fileio.io_controlnetwork import write_filelist
from autocnet.graph.network import CandidateGraph
from autocnet.fileio import utils

#%pylab inline
%pylab qt4

In [ ]:
serial_numbers = {'AS15-M-0295_SML.png': '1971-07-31T01:24:11.754',
                'AS15-M-0296_SML.png': '1971-07-31T01:24:36.970',
                'AS15-M-0297_SML.png': '1971-07-31T01:25:02.243',
                'AS15-M-0298_SML.png': '1971-07-31T01:25:27.457',
                'AS15-M-0299_SML.png': '1971-07-31T01:25:52.669',
                'AS15-M-0300_SML.png': '1971-07-31T01:26:17.923'}

for k, v in serial_numbers.items():
    serial_numbers[k] = 'APOLLO15/METRIC/{}'.format(v)

 

In [ ]:
adjacency = get_path('two_image_adjacency.json')
basepath = get_path('Apollo15')
cg = CandidateGraph.from_adjacency(adjacency, basepath=basepath)

cg.extract_features(method='sift', extractor_parameters={"nfeatures":500})
cg.match_features(k=2)

In [ ]:
for source, destination, edge in cg.edges_iter(data=True):
    # Perform the symmetry check
    edge.symmetry_check()
    # Perform the ratio test
    edge.ratio_check(clean_keys=['symmetry'])

In [ ]:
cg.apply_func_to_edges("compute_fundamental_matrix", clean_keys=['symmetry', 'ratio'])

In [ ]:
f = cg.edge[0][1].fundamental_matrix
f

In [ ]:
len((f.x2.y.values - f.x1.y.values)/(f.x2.x.values - f.x1.x.values))

In [ ]:
df=utils.calculate_slope(f.x1, f.x2)
df.values.ravel()

In [ ]:
f.mask[f.mask==True]

In [ ]:
f.refine(df=utils.calculate_slope(f.x1, f.x2))

In [ ]:
f.rollback()

In [ ]:
f.mask

In [ ]:
figsize(18,18)
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][1].plot(clean_keys=['ratio', 'fundamental'], ax=ax)

In [ ]:
figsize(18,18)
fig, ax = plt.subplots(1,1)
ax = cg.edge[0][2].plot(clean_keys=['ratio', 'symmetry', 'fundamental', 'subpixel'], ax=ax)

In [ ]:

        for source, destination, edge in cg.edges_iter(data=True):

            # Perform the symmetry check
            edge.symmetry_check()
            self.assertIn(edge.masks['symmetry'].sum(), range(400, 600))

            # Perform the ratio test
            edge.ratio_check(clean_keys=['symmetry'])
            self.assertIn(edge.masks['ratio'].sum(), range(40, 100))

        # Step: Compute the homographies and apply RANSAC
        cg.apply_func_to_edges("compute_homography", clean_keys=['symmetry', 'ratio'])

        # Step: Compute the overlap ratio and coverage ratio
        for s, d, edge in cg.edges_iter(data=True):
            edge.coverage_ratio(clean_keys=['symmetry', 'ratio'])

        # Step: Compute subpixel offsets for candidate points
        cg.apply_func_to_edges("subpixel_register", clean_keys=['ransac'])

        # Step: And create a C object
        cnet = cg.to_cnet(clean_keys=['symmetry', 'ratio', 'ransac', 'subpixel'])

        # Step: Create a fromlist to go with the cnet and write it to a file
        filelist = cg.to_filelist()
        write_filelist(filelist, path="fromlis.lis")

        # Step update the serial numbers
        nid_to_serial = {}
        for i, node in cg.nodes_iter(data=True):
            nid_to_serial[i] = self.serial_numbers[node.image_name]

        cnet.replace({'nid': nid_to_serial}, inplace=True)
        # Step: Output a control network
        to_isis('TestTwoImageMatching.net', cnet, mode='wb',
                networkid='TestTwoImageMatching', targetname='Moon')